## Word Pairs

In [103]:
import numpy as np
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [104]:
f_0 = open('/content/drive/My Drive/Sim_Lex999.txt', 'r')
text = f_0.readlines()
print(len(text))
#There are 1000 lines in the text file

1000


In [0]:
row_index = list()
#The last diigt of my UNIQUE ID is 8.
i = 8
while i <= 998:
    row_index.append(i)
    i += 10

In [0]:
word_pairs = list()
for index in row_index:
    word_pairs.append(text[index].split(':')[0:2])

## Word2Vec

In [107]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-26 19:51:31--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   674KB/s    in 46s     

2020-01-26 19:52:17 (665 KB/s) - ‘text8.zip’ saved [31344016/31344016]



In [108]:
!unzip text8.zip

Archive:  text8.zip
  inflating: text8                   


In [109]:
!pip install gensim  

In [0]:
from gensim.models import word2vec

In [111]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8', sg=0, size=50, window=8, negative=25, hs=0, sample=1e-4, workers=20, min_count=5, iter=15)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
vocab = model.wv.vocab

In [114]:
cosine_similarity_score = list()

for i in range(len(word_pairs)):
  if word_pairs[i][0] in vocab and word_pairs[i][1] in vocab:
    word1 = model[word_pairs[i][0]].reshape(1, -1)
    word2 = model[word_pairs[i][1]].reshape(1, -1)
    cosine_similarity_score.append(cosine_similarity(word1, word2))
  elif word_pairs[i][0] not in vocab or word_pairs[i][1] not in vocab:
    cosine_similarity_score.append('N/A')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
/usr/local/lib/python3.6/dist-packages/ip

In [115]:
cosine_similarity_score = np.array(cosine_similarity_score)
cosine_similarity_score = list(cosine_similarity_score)
word2vec_scores = list()
for thing in cosine_similarity_score:
  if thing == 'N/A':
    word2vec_scores.append(thing)
  else:
    word2vec_scores.append(float(thing))
print(len(word2vec_scores))

100


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """


In [116]:
print(len(word_pairs))

100


In [0]:
simlex999_scores = list()
for index in row_index:
    simlex999_scores.append(text[index].split(':')[2])

In [118]:
print(len(simlex999_scores))

100


In [154]:
word1list = list()
word2list = list()
for i in range(len(word_pairs)):
  word1list.append(word_pairs[i][0])
  word2list.append(word_pairs[i][1])
print(len(word1list))
print(len(word2list))

100
100


In [0]:
import pandas as pd

d_word2vec = {'word1': word1list, 'word2': word2list, 'Sim_Lex_Similarity': simlex999_scores, 'Word2Vec_Similarity': word2vec_scores}
df_word2vec = pd.DataFrame(data = d_word2vec)

In [156]:
print(df_word2vec)
df_word2vec.to_csv('/content/drive/My Drive/word2vec.csv')

          word1      word2 Sim_Lex_Similarity Word2Vec_Similarity
0         short       long               1.23             0.97376
1         happy        mad               0.95            0.997754
2        narrow      broad               1.18            0.999025
3     wonderful      great               8.05             0.96331
4   unnecessary  necessary               0.63             0.97582
..          ...        ...                ...                 ...
95        leave     wander               3.57            0.994639
96      collect       save               6.67            0.997904
97          add      build               4.92            0.990119
98  investigate     pursue               7.15          -0.0189734
99       absorb   withdraw               2.97            0.989369

[100 rows x 4 columns]


## Universal Encoder

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [0]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

messages = list()
# Compute a representation for each message, showing various lengths supported.
for i in range(len(word_pairs)):
  messages.append(word_pairs[i][0])
  messages.append(word_pairs[i][1])
word_vector = []

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    #print("Message: {}".format(messages[i]))
    #print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    #print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

In [126]:
print(message_embeddings.shape)
print(len(word_pairs))

(200, 512)
100


In [0]:
index1 = 0
index2 = 1
universal_encoder_scores = list()
while index2 <= 199:
  embedded_word1 = message_embeddings[index1].reshape(1, -1)
  embedded_word2 = message_embeddings[index2].reshape(1, -1)
  similarity_score = cosine_similarity(embedded_word1, embedded_word2)
  universal_encoder_scores.append(float(similarity_score))
  index1 += 2
  index2 += 2

In [128]:
print(len(universal_encoder_scores))

100


In [0]:
d_universal_encoder = {'word1': word1list, 'word2': word2list, 'Sim_Lex_Similarity': simlex999_scores, 'Universal_encoder_Similarity': universal_encoder_scores}
df_universal_encoder = pd.DataFrame(data=d_universal_encoder)

In [160]:
print(df_universal_encoder)
df_universal_encoder.to_csv('/content/drive/My Drive/universal_encoder.csv')

          word1      word2 Sim_Lex_Similarity  Universal_encoder_Similarity
0         short       long               1.23                      0.873932
1         happy        mad               0.95                      0.600090
2        narrow      broad               1.18                      0.649658
3     wonderful      great               8.05                      0.898812
4   unnecessary  necessary               0.63                      0.819382
..          ...        ...                ...                           ...
95        leave     wander               3.57                      0.582286
96      collect       save               6.67                      0.548759
97          add      build               4.92                      0.397505
98  investigate     pursue               7.15                      0.532359
99       absorb   withdraw               2.97                      0.475173

[100 rows x 4 columns]


## Grokking

In [131]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
!cat reviews.txt | head

--2020-01-26 20:11:07--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt.2’

reviews.txt.2       100%[===================>]  32.12M  90.8MB/s    in 0.4s    

2020-01-26 20:11:09 (90.8 MB/s) - ‘reviews.txt.2’ saved [33678267/33678267]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of th

In [132]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -15

--2020-01-26 20:11:12--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt.2’

labels.txt.2        100%[===================>] 219.73K  --.-KB/s    in 0.02s   

2020-01-26 20:11:13 (14.1 MB/s) - ‘labels.txt.2’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive


In [0]:
def pretty_print_review_and_label(i):
   print(labels[i] + "\t:\t" + reviews[i][:80] + "...")

g = open('reviews.txt','r') # What we know!
reviews = list(map(lambda x:x[:-1],g.readlines()))
g.close()

g = open('labels.txt','r') # What we WANT to know!
labels = list(map(lambda x:x[:-1].upper(),g.readlines()))
g.close()

In [134]:
import sys,random,math
from collections import Counter
import numpy as np

np.random.seed(1)
random.seed(1)
f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

print(raw_reviews[1])

story of a man who has unnatural feelings for a pig . starts out with a opening scene that is a terrific example of absurd comedy . a formal orchestra audience is turned into an insane  violent mob by the crazy chantings of it  s singers . unfortunately it stays absurd the whole time with no general narrative eventually making it just too off putting . even those from the era should be turned off . the cryptic dialogue would make shakespeare seem easy to a third grader . on a technical level it  s better than you might think with some good cinematography by future great vilmos zsigmond . future stars sally kirkland and frederic forrest can be seen briefly .  



In [135]:
tokens = list(map(lambda x:(x.split(" ")),raw_reviews))
wordcnt = Counter()
for sent in tokens:
    for word in sent:
        wordcnt[word] -= 1
vocab = list(set(map(lambda x:x[0],wordcnt.most_common())))

print(len(vocab))

74075


In [0]:
word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

In [0]:
concatenated = list()
input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
            concatenated.append(word2index[word])
        except:
            ""
    input_dataset.append(sent_indices)
concatenated = np.array(concatenated)
random.shuffle(input_dataset)

In [0]:
alpha, iterations = (0.05, 2)
hidden_size,window,negative = (50,2,5)  

weights_0_1 = (np.random.rand(len(vocab),hidden_size) - 0.5) * 0.2
weights_1_2 = np.random.rand(len(vocab),hidden_size)*0

layer_2_target = np.zeros(negative+1)
layer_2_target[0] = 1

In [0]:
def similar(target='beautiful'):
  target_index = word2index[target]

  scores = Counter()
  for word,index in word2index.items():
    raw_difference = weights_0_1[index] - (weights_0_1[target_index])
    squared_difference = raw_difference * raw_difference
    scores[word] = -math.sqrt(sum(squared_difference))
  return scores.most_common(10)

In [0]:
def sigmoid(x):
    return 1/(1 + np.exp(-x))

In [141]:
for rev_i,review in enumerate(input_dataset * iterations):
  for target_i in range(len(review)):
        
    # since it's really expensive to predict every vocabulary
    # we're only going to predict a random subset
    target_samples = [review[target_i]]+list(concatenated\
    [(np.random.rand(negative)*len(concatenated)).astype('int').tolist()])

    left_context = review[max(0,target_i-window):target_i]
    right_context = review[target_i+1:min(len(review),target_i+window)]

    layer_1 = np.mean(weights_0_1[left_context+right_context],axis=0)
    layer_2 = sigmoid(layer_1.dot(weights_1_2[target_samples].T))
    layer_2_delta = layer_2 - layer_2_target
    layer_1_delta = layer_2_delta.dot(weights_1_2[target_samples])

    weights_0_1[left_context+right_context] -= layer_1_delta * alpha
    weights_1_2[target_samples] -= np.outer(layer_2_delta,layer_1)*alpha

  if(rev_i % 1000 == 0):
    sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)
        *iterations)) + "   " + str(similar('terrible')))
    #sys.stdout.write('\rProgress:'+str(rev_i/float(len(input_dataset)*iterations)))
    #print(similar('terrible'))

Progress:0.98   [('terrible', -0.0), ('horrible', -2.94463577739708), ('superb', -3.409103318623392), ('brilliant', -3.7544073805852274), ('miserable', -3.7830419455697566), ('phenomenal', -3.791813306400485), ('fantastic', -3.8211741992132873), ('pathetic', -3.828072934490683), ('masterful', -3.8287367578637173), ('magnificent', -3.872633217080507)]

In [0]:
temp = word2index.items()

In [143]:
test_vocab = list()
for thing in temp:
  test_vocab.append(thing[0])
print(len(test_vocab))

74075


In [144]:
print(len(word_pairs))
word_index = list()
for i in range(len(word_pairs)):
  if word_pairs[i][0] in test_vocab and word_pairs[i][1] in test_vocab:
    pair_index = list()
    pair_index.append(word2index[word_pairs[i][0]])
    pair_index.append(word2index[word_pairs[i][1]])
    word_index.append(pair_index)
  elif word_pairs[i][0] not in test_vocab or word_pairs[i][1] not in test_vocab:
    word_index.append('N/A')

100


In [145]:
print(len(word_index))

100


In [0]:
from sklearn.metrics.pairwise import cosine_similarity

grokking_scores = list()
for thing in word_index:
  if thing == 'N/A':
    grokking_scores.append('N/A')
  else:
    embed_word1 = weights_0_1[thing[0]].reshape(1, -1)
    embed_word2 = weights_0_1[thing[1]].reshape(1, -1)
    similarity_score = cosine_similarity(embed_word1, embed_word2)
    grokking_scores.append(float(similarity_score))

In [147]:
print(len(grokking_scores))

100


In [0]:
d_grokking = {'word1': word1list, 'word2': word2list, 'Sim_Lex_Similarity': simlex999_scores, 'Universal_encoder_Similarity': grokking_scores}
df_grokking = pd.DataFrame(data=d_grokking)

In [158]:
print(df_grokking)
df_grokking.to_csv('/content/drive/My Drive/grokking.csv')

          word1      word2 Sim_Lex_Similarity Universal_encoder_Similarity
0         short       long               1.23                     0.414243
1         happy        mad               0.95                     0.267115
2        narrow      broad               1.18                     0.706805
3     wonderful      great               8.05                     0.745793
4   unnecessary  necessary               0.63                     0.287326
..          ...        ...                ...                          ...
95        leave     wander               3.57                     0.459421
96      collect       save               6.67                     0.717075
97          add      build               4.92                     0.706693
98  investigate     pursue               7.15                      0.71641
99       absorb   withdraw               2.97                     0.575652

[100 rows x 4 columns]
